<a href="https://colab.research.google.com/github/tatianamgar/Work_GPN/blob/main/BERT_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import numpy as np
from tqdm import tqdm, trange

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig
from transformers import BertForTokenClassification, AdamW

In [ ]:
model = BertForTokenClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=38,
    output_attentions = False,
    output_hidden_states = False
)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

In [ ]:
model.load_state_dict(torch.load('drive/My Drive/NER BERT fine_tune/bert_model_2.pth'))
model.eval()
model.cuda()

In [ ]:
tag_values = ['B-исполнение пожарной опасности',
 'B-исполнение',
 'B-материал изоляции',
 'I-исполнение пожарной опасности',
 'I-кол-во жил',
 'I-материал жил',
 'B-Вид изделия',
 'B-тип кабеля',
 'B-номер стандарта',
 'I-тип опоры по конструкции',
 ' B-производитель',
 'I-сечение жил',
 'B-тех характеристики',
 'I-материал',
 'I-диаметр трубопровода',
 'B-толщина трубопровода',
 'I-толщина трубопровода',
 'B-производитель',
 'I-исполнение',
 'B-кол-во жил',
 'B-марка кабеля',
 'B-диаметр трубопровода',
 'B-тип опоры',
 'B-материал жил',
 'B-климатическое исполнение',
 'I-усиление',
 'B-марка стали изделия',
 'B-тип опоры по конструкции',
 'B-материал',
 'I-материал изоляции',
 'B-НД основной',
 'O',
 'B-сечение жил',
 'B-усиление',
 'I-НД основной',
 'B-вид изделия',
 'I-марка стали изделия',
 'PAD']

In [ ]:
def make_prediction(test_sentence):
  tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
  tokenized_sentence = tokenizer.encode(test_sentence)
  input_ids = torch.tensor([tokenized_sentence]).cuda()
  with torch.no_grad():
    output = model(input_ids)
  label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)  
  # join bpe split tokens
  tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
  new_tokens, new_labels = [], []
  for token, label_idx in zip(tokens, label_indices[0]):
      if token.startswith("##"):
          new_tokens[-1] = new_tokens[-1] + token[2:]
      else:
          new_labels.append(tag_values[label_idx])
          new_tokens.append(token)
  
  for token, label in zip(new_tokens, new_labels):
    print("{}\t{}".format(label, token))

In [ ]:
test_sentence = """
Кабель контрольный медный, с изоляцией и в оболочке из ПВХ пластиката, огнестойкий, пониженной горючести с низким дымо- и газовыделением, 
с защитным покровом типа БбШв, состоящий из 4 жил сечением 2,5 мм2, КВБбШвнг(А)-FRLS 4х2,5
"""

make_prediction(test_sentence)

In [ ]:
test_sentence = """
Опора корпусная 
"""

make_prediction(test_sentence)

B-вид изделия	[CLS]
B-вид изделия	Опора
B-тип опоры по конструкции	корпусная
B-вид изделия	[SEP]


In [ ]:
test_sentence = """
Опора бугельная БКХЛ 100.57 ТУ 146882-01-00158675-2003
"""

make_prediction(test_sentence)

B-вид изделия	[CLS]
B-вид изделия	Опора
B-тип опоры по конструкции	бугельная
B-тип опоры	БКХЛ
O	100
O	.
O	57
B-НД основной	ТУ
I-НД основной	146882
I-НД основной	-
I-НД основной	01
I-НД основной	-
I-НД основной	00158675
I-НД основной	-
I-НД основной	2003
B-вид изделия	[SEP]
